In [27]:
import numpy as np
import pandas as pd
import requests
import json
import psycopg2
from sqlalchemy import create_engine
import os

In [3]:
propublica_token = os.environ['propublica_token']
postgres_password = os.environ['POSTGRES_PASSWORD']

In [4]:
import getdata

In [5]:
members_vv, cvote_vv, memvotes_vv = getdata.get_voteview()

In [10]:
useragent = getdata.get_useragent()

In [12]:
members_pp = getdata.get_propublica(propublica_token, useragent, email='jkropko@virginia.edu')

In [14]:
members = getdata.merge_members(members_pp=members_pp, members_vv=members_vv)

In [16]:
members.head(1).T

,0
title,Representative
short_title,Rep.
first_name,Alma
middle_name,None
last_name,ADAMS
suffix,None
congress,117.0
chamber,House
icpsr,21545.0
state,NC


In [22]:
members['propublica_id'].isna().value_counts()

False    556
Name: propublica_id, dtype: int64

In [18]:
cvote_vv.head(1).T

,0
congress,117
chamber,House
rollnumber,1
date,2021-01-03
session,1
clerk_rollnumber,2
yea_count,216
nay_count,211
nominate_mid_1,-0.057
nominate_mid_2,0.038


In [19]:
memvotes_vv.head(1).T

,0
congress,117
chamber,House
rollnumber,1
icpsr,14066.0
cast_code,6
prob,100.0


# Initialize Postgres

In [24]:
server = psycopg2.connect(
    user = 'postgres',
    password = postgres_password,
    host = 'postgres',
    port = '5432')
server.autocommit = True

In [25]:
cursor = server.cursor()

In [26]:
try:
    cursor.execute("CREATE DATABASE contrans")
except:
    cursor.execute("DROP DATABASE contrans")
    cursor.execute("CREATE DATABASE contrans")

## Add data to contrans DB

In [28]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = postgres_password,
    host = 'postgres',
    port = '5432',
    db = 'contrans'))

In [30]:
members.to_sql('members', con=engine, if_exists='replace', index=False, chunksize=1000)
memvotes_vv.to_sql('member_vote', con=engine, if_exists='replace', index=False, chunksize=1000)
cvote_vv.to_sql('rollcalls', con=engine, if_exists='replace', index=False, chunksize=1000)

1723

## Example queries

In [33]:
myquery = '''
SELECT * FROM rollcalls
LIMIT 5
'''
pd.read_sql(myquery, con=engine)

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
0,117,House,1,2021-01-03,1,2,216,211,-0.057,0.038,0.707,0.274,-4.069,None,Pelosi,None,Election of the Speaker,None
1,117,House,2,2021-01-03,1,3,371,2,0.000,0.000,0.000,0.000,0.000,HRES1,Passed,Authorizing and directing the Speaker to admin...,On Agreeing to the Resolution,None
2,117,House,3,2021-01-04,1,4,214,204,0.019,-0.125,0.711,-0.235,-0.242,HRES8,Passed,Adopting the Rules of the House of Representat...,On Motion to Table the Motion to Postpone to a...,None
3,117,House,4,2021-01-04,1,5,214,196,0.020,-0.133,0.710,-0.240,-0.240,HRES8,Passed,Adopting the Rules of the House of Representat...,Table Motion to Refer,None
4,117,House,5,2021-01-04,1,6,217,204,0.019,-0.124,0.711,-0.234,-0.242,HRES8,Passed,Adopting the Rules of the House of Representat...,On Ordering the Previous Question,None
